In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime, time
import sklearn, sklearn.metrics, sklearn.preprocessing, sklearn.linear_model, sklearn.ensemble, sklearn.model_selection
import nltk, nltk.stem
import pickle
import collections

import modutils

data_dir = '../DataSets/MercariPrice/'
src_file = data_dir + 'train_title.csv'
stat_file = '21MerPrice04_TextStat.csv'

In [6]:
%%time
src = pd.read_csv(src_file)

Wall time: 11.2 s


In [8]:
with open(stat_file, 'rb') as f:
    wordStat = pickle.load(f)

In [10]:
src[:10]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,...,brand_200,cat3g_100,cat3g_500,cat4g_100,fcst_simple,fcst_log_simple,fcst_abs_simple,fcst_diff_simple,fcst_log_simple_title,fcst_diff_simple_title
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,...,OTHER,Men/Tops/T-shirts,Men/Tops/T-shirts,Men/Tops/T-shirts/NONE,2.569505,2.569505,12.059363,-0.171610,2.272534,0.125362
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,...,OTHER,Electronics/Computers & Tablets/Components & P...,OTHER,Electronics/Computers & Tablets/Components & P...,3.265820,3.265820,25.201577,0.704472,3.229182,0.741110
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,...,Target,Women/Tops & Blouses/Blouse,Women/Tops & Blouses/Blouse,Women/Tops & Blouses/Blouse/NONE,2.624912,2.624912,12.803355,-0.227016,2.525676,-0.127781
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,...,OTHER,Home/Home Décor/Home Décor Accents,Home/Home Décor/Home Décor Accents,Home/Home Décor/Home Décor Accents/NONE,2.737056,2.737056,14.441463,0.846463,2.756941,0.826578
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,...,OTHER,Women/Jewelry/Necklaces,Women/Jewelry/Necklaces,Women/Jewelry/Necklaces/NONE,2.835247,2.835247,16.034600,0.971416,2.722495,1.084168
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Women,Other,...,OTHER,Women/Other/Other,Women/Other/Other,Women/Other/Other/NONE,2.915577,2.915577,17.459459,1.178768,3.418110,0.676235
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,Women,Swimwear,...,OTHER,Women/Swimwear/Two-Piece,Women/Swimwear/Two-Piece,Women/Swimwear/Two-Piece/NONE,2.886619,2.886619,16.932571,1.287769,2.763533,1.410854
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,Sports & Outdoors,Apparel,...,OTHER,Sports & Outdoors/Apparel/Girls,OTHER,Sports & Outdoors/Apparel/Girls/NONE,2.502017,2.502017,11.207095,-0.556107,2.957642,-1.011731
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,...,Nike,Sports & Outdoors/Apparel/Girls,OTHER,Sports & Outdoors/Apparel/Girls/NONE,2.823175,2.823175,15.830205,0.172557,2.862557,0.133175
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0,I realized his pants are on backwards after th...,Vintage & Collectibles,Collectibles,...,OTHER,Vintage & Collectibles/Collectibles/Doll,OTHER,Vintage & Collectibles/Collectibles/Doll/NONE,2.974752,2.974752,18.584758,-0.777527,2.900488,-0.703264
